In [5]:
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = json.loads(open('words_data.json').read())
classes = json.loads(open('classes_data.json').read())

In [7]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               7936      
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 455       
Total params: 16,647
Trainable params: 16,647
Non-trainable params: 0
_________________________________________________________________


In [9]:
print(words)
print(classes)

['a', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'introduce', 'is', 'later', 'me', 'money', 'much', 'name', 'newest', 'next', 'novel', 'order', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'what', 'who', 'you', 'your', 'yourself']
['buybooks', 'goodbye', 'greeting', 'introduce', 'noanswer', 'payment', 'thanks']


In [11]:
def clean(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenize the pattern
    sentence_words = clean(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
#     print(res)

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [13]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    predict_intent = ints[0]['intent']
    # print(ints)
    res = getResponse(ints, intents)
    
    return res, ints[0]['intent']

In [15]:
# while(True):
#     w = str(input())
#     print("You: ", w)
#     res = chatbot_response(w)
#     print("Bot: ",res)
    

## Create data for Online Shopping

In [21]:
import pandas as pd
import numpy as np

data = pd.read_csv("books.csv")

arr = np.array(data)
# print(arr)

In [22]:
products = []
for i in range(len(arr)):
    pd = {
        "id": arr[i][0],
        "book": arr[i][1],
        "author": arr[i][2],
        "price": arr[i][3]
    }
    print(pd)
    products.append(pd)
products = json.dumps(products)
products

{'id': 101, 'book': 'The Lord of the Rings', 'author': 'J.R.R. Tolkien', 'price': 599999}
{'id': 102, 'book': 'The Kite Runner', 'author': 'Khaled Hosseini', 'price': 649999}
{'id': 103, 'book': 'Harry Potter and the Philosopher Stone', 'author': 'J.K. Rowling', 'price': 549999}
{'id': 104, 'book': 'Slaughterhouse-Five', 'author': 'Kurt Vonnegut', 'price': 549999}
{'id': 105, 'book': 'The Lion, the Witch, and the Wardrobe', 'author': 'C.S. Lewis', 'price': 549999}
{'id': 106, 'book': 'To Kill a Mockingbird', 'author': 'Harper Lee', 'price': 599999}
{'id': 107, 'book': 'The Book Thief', 'author': 'Markus Zusak', 'price': 499999}
{'id': 108, 'book': 'Wuthering Heights', 'author': 'Emily Bronte', 'price': 649999}
{'id': 109, 'book': 'The Catcher in the Rye', 'author': 'J.D. Salinger', 'price': 449999}
{'id': 110, 'book': 'Animal Farm', 'author': 'George Orwell', 'price': 499999}


'[{"id": 101, "book": "The Lord of the Rings", "author": "J.R.R. Tolkien", "price": 599999}, {"id": 102, "book": "The Kite Runner", "author": "Khaled Hosseini", "price": 649999}, {"id": 103, "book": "Harry Potter and the Philosopher Stone", "author": "J.K. Rowling", "price": 549999}, {"id": 104, "book": "Slaughterhouse-Five", "author": "Kurt Vonnegut", "price": 549999}, {"id": 105, "book": "The Lion, the Witch, and the Wardrobe", "author": "C.S. Lewis", "price": 549999}, {"id": 106, "book": "To Kill a Mockingbird", "author": "Harper Lee", "price": 599999}, {"id": 107, "book": "The Book Thief", "author": "Markus Zusak", "price": 499999}, {"id": 108, "book": "Wuthering Heights", "author": "Emily Bronte", "price": 649999}, {"id": 109, "book": "The Catcher in the Rye", "author": "J.D. Salinger", "price": 449999}, {"id": 110, "book": "Animal Farm", "author": "George Orwell", "price": 499999}]'

## Build the User Interface

In [23]:
from flask import Flask, render_template, request, make_response
from flask_cors import CORS

In [ ]:
app = Flask(__name__)

CORS(app)
app.config.update(
    TEMPLATES_AUTO_RELOAD = True
)
path = "main.html"

@app.route("/")
def home_page():
    return (render_template("main.html"))

@app.route('/products')
def product_page():
    return (render_template("products.html"))

@app.route("/payment")
def payment_page():
    return render_template("payment.html")

@app.route("/get_products")
def get_products():  
    res = make_response(products)
#     res.set_cookie('cross-site-cookie', samesite='None', secure=True);
    return res

@app.route("/get_bot")
def bot_response():
    text = request.args.get('msg')
    res = chatbot_response(text) 
    data = json.dumps({"intent": res[1], "response": res[0]})    
    return data


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/May/2021 23:08:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2021 23:09:00] "GET /get_bot?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20hello%3C%2Fspan%3E HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2021 23:09:04] "GET /get_bot?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20thanks%3C%2Fspan%3E HTTP/1.1" 200 -
